In [1]:
# !pip install --upgrade pandas
# !pip install --upgrade google-api-python-client
# !pip install --upgrade seaborn

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io import gbq
from pandas_gbq import read_gbq
import json
import lightgbm as lgb
import gc
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
# from pyspark.sql import functions as sfunc
# from pyspark.sql import types as stypes

In [3]:
!ls

BQ_Connect-FTRL.ipynb
ftrl_submission.csv
Kaggle-TalkingData-f0e6b0d0e656.json
LGBM_99.ipynb
LGBM_v1.ipynb
LGBM_v2.ipynb
sub_lgbm_r_to_python_nocv_150upsample.csv
sub_lgbm_r_to_python_nocv_170upsample_maxdepth.csv
sub_lgbm_r_to_python_nocv.csv
sub_xgb_nocv_150upsample.csv
test.csv
train.csv
Untitled.ipynb
Valid-data_generation.ipynb
Wordbatch.ipynb
XGB_v1.ipynb


In [4]:
VALIDATE = False

MAX_ROUNDS = 250
EARLY_STOP = 50
OPT_ROUNDS = 250

In [5]:
FULL_OUTFILE = 'xgb_nocv_150upsample_sub.csv'
VALID_OUTFILE = 'xgb_withcv_150upsample_valid.csv'

import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

path = ''

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

print('load train...')
train_cols = ['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
train_df = pd.read_csv(path+"train.csv", dtype=dtypes,usecols=train_cols)

#train_df = pd.read_csv(path+"train.csv", skiprows=range(1,84903891), nrows=100000000,dtype=dtypes, usecols=train_cols)

load train...


In [6]:
path1 = '../valid-train_data/'
valid_df = pd.read_pickle(path1+'validation.pkl.gz')

In [7]:
import gc

len_train = len(train_df)

gc.collect()

print('data prep...')

most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
least_freq_hours_in_test_data = [6, 11, 15]


def prep_data( df ):
    
    df['hour'] = pd.to_datetime(df.click_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(df.click_time).dt.day.astype('uint8')
    df.drop(['click_time'], axis=1, inplace=True)
    gc.collect()
    
    df['in_test_hh'] = (   3 
                         - 2*df['hour'].isin(  most_freq_hours_in_test_data ) 
                         - 1*df['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')
    print( df.info() )

    print('group by : ip_day_test_hh')
    gp = df[['ip', 'day', 'in_test_hh', 'channel']].groupby(by=['ip', 'day',
             'in_test_hh'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_day_test_hh'})
    df = df.merge(gp, on=['ip','day','in_test_hh'], how='left')
    del gp
    df.drop(['in_test_hh'], axis=1, inplace=True)
    print( "nip_day_test_hh max value = ", df.nip_day_test_hh.max() )
    df['nip_day_test_hh'] = df['nip_day_test_hh'].astype('uint32')
    gc.collect()
    print( df.info() )

    print('group by : ip_day_hh')
    gp = df[['ip', 'day', 'hour', 'channel']].groupby(by=['ip', 'day', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_day_hh'})
    df = df.merge(gp, on=['ip','day','hour'], how='left')
    del gp
    print( "nip_day_hh max value = ", df.nip_day_hh.max() )
    df['nip_day_hh'] = df['nip_day_hh'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_os')
    gp = df[['ip', 'day', 'os', 'hour', 'channel']].groupby(by=['ip', 'os', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_os'})
    df = df.merge(gp, on=['ip','os','hour','day'], how='left')
    del gp
    print( "nip_hh_os max value = ", df.nip_hh_os.max() )
    df['nip_hh_os'] = df['nip_hh_os'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_app')
    gp = df[['ip', 'app', 'hour', 'day', 'channel']].groupby(by=['ip', 'app', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_app'})
    df = df.merge(gp, on=['ip','app','hour','day'], how='left')
    del gp
    print( "nip_hh_app max value = ", df.nip_hh_app.max() )
    df['nip_hh_app'] = df['nip_hh_app'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_dev')
    gp = df[['ip', 'device', 'hour', 'day', 'channel']].groupby(by=['ip', 'device', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_dev'})
    df = df.merge(gp, on=['ip','device','day','hour'], how='left')
    del gp
    print( "nip_hh_dev max value = ", df.nip_hh_dev.max() )
    df['nip_hh_dev'] = df['nip_hh_dev'].astype('uint32')
    gc.collect()
    print( df.info() )

    df.drop( ['ip','day'], axis=1, inplace=True )
    gc.collect()
    print( df.info() )
    
    return( df )

#---------------------------------------------------------------------------------

print( "Train info before: ")
print( train_df.info() )
train_df = prep_data( train_df )
gc.collect()
print( "Train info after: ")
print( train_df.info() )

print("vars and data type: ")
train_df.info()

data prep...
Train info before: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       object
is_attributed    uint8
dtypes: object(1), uint16(4), uint32(1), uint8(1)
memory usage: 3.6+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 9 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
is_attributed    uint8
hour             uint8
day              uint8
in_test_hh       uint8
dtypes: uint16(4), uint32(1), uint8(4)
memory usage: 2.8 GB
None
group by : ip_day_test_hh
nip_day_test_hh max value =  208661
<class 'pandas.core.frame.DataFrame'>
Int64Index: 184903890 entries, 0 to 184903889
Data columns (total 9 columns):
ip                 uint32
app       

In [8]:
metrics = 'auc'
xgb_params = {'eta': 0.1,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 4, 
          'subsample': 0.7, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':170,
          'eval_metric': 'auc', 
          'nthread':-1,
          'random_state': 99, 
          'silent': True}

target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour', 'nip_day_test_hh', 'nip_day_hh',
              'nip_hh_os', 'nip_hh_app', 'nip_hh_dev']
categorical = ['app', 'device', 'os', 'channel', 'hour']

print(train_df.head(5))

   app  device  os  channel  is_attributed  hour  nip_day_test_hh  nip_day_hh  \
0    3       1  13      379              0    14                1           1   
1    3       1  19      379              0    14                1           1   
2    3       1  13      379              0    14                1           1   
3   14       1  13      478              0    14                1           1   
4    3       1  13      379              0    14                1           1   

   nip_hh_os  nip_hh_app  nip_hh_dev  
0          1           1           1  
1          1           1           1  
2          1           1           1  
3          1           1           1  
4          1           1           1  


In [9]:
if VALIDATE:

    train_df, val_df = train_test_split( train_df, train_size=.95, shuffle=False )

    print(train_df.info())
    print(val_df.info())

    print("train size: ", len(train_df))
    print("valid size: ", len(val_df))

    gc.collect()

    print("Training...")

    num_boost_round=MAX_ROUNDS
    early_stopping_rounds=EARLY_STOP

    xgtrain = xgb.DMatrix(train_df[predictors].values, label=train_df[target].values)
    del train_df
    gc.collect()

    xgvalid = xgb.DMatrix(val_df[predictors].values, label=val_df[target].values)
    del val_df
    gc.collect()

    evals_results = {}
    watchlist = [(xgtrain, 'train'), (xgvalid, 'valid')]
    #model = xgb.train(params, dtrain, 200, watchlist, maximize=True, early_stopping_rounds = 25, verbose_eval=5)
    
    bst = xgb.train(xgb_params, 
                     xgtrain, 
                    250,
                    watchlist,
                    maximize=True,
#                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10)
    
    n_estimators = bst.best_iteration

    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
    
    outfile = VALID_OUTFILE
    
    del xgvalid

else:

    print(train_df.info())

    print("train size: ", len(train_df))

    gc.collect()

    print("Training...")

    num_boost_round=OPT_ROUNDS

    xgtrain = xgb.DMatrix(train_df[predictors].values, label=train_df[target].values)
    del train_df
    gc.collect()
    watchlist = [(xgtrain, 'train')]
    bst = xgb.train(xgb_params, 
                     xgtrain, 
                    250,
                    watchlist,
                    maximize=True,
#                    early_stopping_rounds=early_stopping_rounds,
                    verbose_eval=10)
                         
    outfile = FULL_OUTFILE
    valid_file = VALID_OUTFILE
del xgtrain
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184903890 entries, 0 to 184903889
Data columns (total 11 columns):
app                uint16
device             uint16
os                 uint16
channel            uint16
is_attributed      uint8
hour               uint8
nip_day_test_hh    uint32
nip_day_hh         uint16
nip_hh_os          uint16
nip_hh_app         uint16
nip_hh_dev         uint32
dtypes: uint16(7), uint32(2), uint8(2)
memory usage: 5.5 GB
None
train size:  184903890
Training...
[0]	train-auc:0.910955
[10]	train-auc:0.95548
[20]	train-auc:0.957234
[30]	train-auc:0.96179
[40]	train-auc:0.964538
[50]	train-auc:0.966727
[60]	train-auc:0.96793
[70]	train-auc:0.969061
[80]	train-auc:0.969911
[90]	train-auc:0.970625
[100]	train-auc:0.971358
[110]	train-auc:0.971823


KeyboardInterrupt: 

In [ ]:
print('load test...')
test_cols = ['ip','app','device','os', 'channel', 'click_time', 'click_id']
test_df = pd.read_csv(path+"test.csv", dtype=dtypes, usecols=test_cols)

test_df = prep_data( test_df )
valid_df = prep_data( valid_df )
gc.collect()

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']

print("Predicting...")
aa = xgb.DMatrix(test_df[predictors].values)
sub['is_attributed'] = bst.predict(aa)
sub.head()
print("writing...")
sub.to_csv(path1+outfile, index=False, float_format='%.9f')
print("done...")
print(sub.info())

In [ ]:
val = pd.DataFrame()
bb = xgb.DMatrix(valid_df[predictors].values)
val['is_attributed'] = bst.predict(bb)
val.to_csv(path1+valid_file, index=False, float_format='%.9f')

In [ ]:
!ls